In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard library imports.
import datetime
import itertools
import json

# Related third party imports.
import numpy as np
import pandas as pd
import tqdm

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
from src.data.json_encoder import NumpyEncoder

In [3]:
network_name = '25_italy'
max_vSDN_size = 5
hp_type = 'ilp'
hp_objective = 'acceptance ratio'

possible_settings = {
    'network_name': [network_name],
    'latency_factor': np.arange(0.6, 0.8, 0.1),
    'shortest_k': [16],
    'hp_type': [hp_type],
    'hp_objective': [hp_objective],
    'hp_repeat':[3],
    'repeat': [100],
    'max_request_size': [max_vSDN_size]
}
param_names_1 = list(possible_settings.keys())
setting_generator = [
    dict(zip(param_names_1, x))
    for x in itertools.product(*possible_settings.values())
]

possible_request_settings = {
    'request_size': range(2, max_vSDN_size),
    'count': [100]
}
# param_names_2 = list(possible_request_settings.keys())
# request_setting_generator = [
#     dict(zip(param_names_2, x))
#     for x in itertools.product(*possible_request_settings.values())
# ]

In [ ]:
# Run
setting = setting_generator[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_simulation(possible_request_settings=possible_request_settings, **setting)
run_log = ns.get_logs()

In [ ]:
# run_log = ns.log_simulation()
run_log

In [ ]:
# Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.run_simulation(possible_request_settings=possible_request_settings, **setting)
    simulation_logs.extend(ns.get_logs())

In [ ]:
# Baseline Run
setting = setting_generator[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.baseline_simulation(possible_request_settings=possible_request_settings, **setting)
run_log = ns.get_logs()

In [37]:
# Baseline Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.baseline_simulation(possible_request_settings=possible_request_settings, **setting)
    simulation_logs.extend(ns.get_logs())

  0%|          | 0/3 [00:00<?, ?it/s]

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 76 rows, 10284 columns and 3099 nonzeros
Model fingerprint: 0xe93c8f26
Model has 9059 general constraints
Variable types: 0 continuous, 10284 integer (10284 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e-02, 3e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve added 5784 rows and 0 columns
Presolve removed 0 rows and 6237 columns
Presolve time: 0.32s
Presolved: 5860 rows, 4047 columns, 36383 nonzeros
Variable types: 0 continuous, 4047 integer (4047 binary)
Found heuristic solution: objective 0.8529412

Root relaxation: objective 1.000000e+00, 3394 iterations, 0.51 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       1.0000

 33%|███▎      | 1/3 [05:12<10:24, 312.11s/it]

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 76 rows, 10284 columns and 4039 nonzeros
Model fingerprint: 0xe09e8891
Model has 9059 general constraints
Variable types: 0 continuous, 10284 integer (10284 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e-02, 3e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve added 7438 rows and 0 columns
Presolve removed 0 rows and 5137 columns
Presolve time: 0.51s
Presolved: 7514 rows, 5147 columns, 68719 nonzeros
Variable types: 0 continuous, 5147 integer (5147 binary)
Found heuristic solution: objective 0.9117647
Found heuristic solution: objective 0.9411765

Root relaxation: objective 1.000000e+00, 4338 iterations, 0.86 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

 67%|██████▋   | 2/3 [11:26<05:30, 330.79s/it]

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 76 rows, 10284 columns and 4039 nonzeros
Model fingerprint: 0xe09e8891
Model has 9059 general constraints
Variable types: 0 continuous, 10284 integer (10284 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e-02, 3e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve added 7438 rows and 0 columns
Presolve removed 0 rows and 5137 columns
Presolve time: 0.50s
Presolved: 7514 rows, 5147 columns, 68719 nonzeros
Variable types: 0 continuous, 5147 integer (5147 binary)
Found heuristic solution: objective 0.9117647
Found heuristic solution: objective 0.9411765

Root relaxation: objective 1.000000e+00, 4338 iterations, 0.99 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

100%|██████████| 3/3 [17:42<00:00, 354.20s/it]


In [ ]:
with open(f"../results/{network_name}/{datetime.date.today()}-{network_name}-test.json", 'w') as file:
    json.dump(simulation_logs, file, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)

# Analyse simulation logs

In [ ]:
df_heu = pd.DataFrame(simulation_logs)

In [36]:
df_heu.query("latency_factor == 0.6 & vSDN_size == 4")[['hp_runtime', 'acceptance_ratio']].mean()

hp_runtime          6.824
acceptance_ratio    0.657
dtype: float64

In [ ]:
df_ilp = pd.DataFrame(simulation_logs)

In [40]:
df_ilp.query("latency_factor == 0.6 & vSDN_size == 4")[['hp_runtime', 'acceptance_ratio']]#.mean()

,hp_runtime,acceptance_ratio
2,0.72,0.68
5,0.54,0.72
8,0.57,0.66
11,0.59,0.62
14,0.54,0.64
17,0.59,0.72
20,0.59,0.62
23,0.61,0.68
26,0.61,0.68
29,0.54,0.70


In [38]:
df_opt = pd.DataFrame(simulation_logs)

In [39]:
df_opt.query("latency_factor == 0.6 & vSDN_size == 4")[['hp_runtime', 'acceptance_ratio']]#.mean()

,hp_runtime,acceptance_ratio
2,16.16,1.0
5,14.51,1.0
8,35.92,1.0
11,29.04,1.0
14,33.09,1.0
17,28.27,1.0
20,29.09,1.0
23,22.06,1.0
26,18.18,1.0
29,5.40,1.0
